In [ ]:
# Instala o Unsloth e as dependências necessárias para o Colab
!pip install --upgrade pip
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# Instala bibliotecas extras para inferência e futuro RAG
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes
!pip install langchain langchain-community sentence-transformers chromadb

In [ ]:
from unsloth import FastLanguageModel
import torch

model_name = "vitateje/biobyai" # Seu adapter no HF

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = 2048,
    load_in_4bit = True, # Como você usou 4bit no fine-tuning
)

# Ativar inferência rápida
FastLanguageModel.for_inference(model)

# Teste simples
inputs = tokenizer(
    [
        "Explique o conceito de edição genética de forma simples."
    ], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 256)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))